PArt 1: Web Scraping 

In [1]:
import requests
import json
from bs4 import BeautifulSoup
import re
import csv
import pandas as pd

#Store the URL for the wikipedia page
URL = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

#Get the HTML text for the URL
resp = requests.get(URL)
html_code = resp.text


In [2]:
#Instantiate html parser object
soup = BeautifulSoup(html_code)

#Get the first table on the webpage and all rows within it 
table = soup.find_all('table')[0]
table_rows = table.find_all('tr')

In [3]:
#Add rows values to a list and convert it into a data frame
l = []
for tr in table_rows:
    td = tr.find_all('td')
    row = [tr.text for tr in td]
    l.append(row)
df = pd.DataFrame(l, columns=['Postal Code', 'Borough', 'Neighbourhood'])
df = df.iloc[1:]
df = df.replace({'\n':''}, regex=True)

In [5]:
df.head()

,Postal Code,Borough,Neighbourhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,"Regent Park, Harbourfront"


Part 2: Get Lattitude and Longitude for each postal code

In [6]:
!pip install geocoder

     |████████████████████████████████| 102kB 3.8MB/s 


In [7]:
!pip install pgeocode

In [8]:
import pgeocode
nomi = pgeocode.Nominatim('ca')
lat_log= nomi.query_postal_code("M2A")

In [9]:
latitude = []
longitude = []
for i in range(len(df['Postal Code'])):
  coord= nomi.query_postal_code(df['Postal Code'].iloc[i])
  latitude.append(coord.latitude)
  longitude.append(coord.latitude)

df['Latitude'] = latitude
df['Longitude'] = longitude

In [10]:
# Removing all boroughs which are not assigned and removing any rows with missing longitude or latitude data
df = df[df['Borough'] != 'Not assigned']
df = df.dropna(subset=['Latitude', 'Longitude'])

In [12]:
df.head(50)

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
3,M3A,North York,Parkwoods,43.7545,43.7545
4,M4A,North York,Victoria Village,43.7276,43.7276
5,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6555,43.6555
6,M6A,North York,"Lawrence Manor, Lawrence Heights",43.7223,43.7223
7,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.6641,43.6641
9,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.6662,43.6662
10,M1B,Scarborough,"Malvern, Rouge",43.8113,43.8113
12,M3B,North York,Don Mills,43.7450,43.7450
13,M4B,East York,"Parkview Hill, Woodbine Gardens",43.7063,43.7063
14,M5B,Downtown Toronto,"Garden District, Ryerson",43.6572,43.6572


In [13]:
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library
#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests

In [14]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
tor_lat = location.latitude
tor_lon  = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(tor_lat, tor_lon))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [15]:
# create map of toronto using latitude and longitude values
map_toronto = folium.Map(location=[tor_lat, tor_lon], zoom_start=11)

# add markers to map
for lat, lng, label in zip(df['Latitude'], df['Longitude'], df['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Part 3 : Explorinf and clustering data

In [16]:
CLIENT_ID = 'I3C1JCMVYJ5C0RGHF2VVQH31BZY4EOOFNPZ2Z1OM1SUFHAV0' # your Foursquare ID
CLIENT_SECRET = 'VJ3WUI2BTMYCV0EKJNOBUDSMQ43DUFALJYXNJPQPVXF4NSZW' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: I3C1JCMVYJ5C0RGHF2VVQH31BZY4EOOFNPZ2Z1OM1SUFHAV0
CLIENT_SECRET:VJ3WUI2BTMYCV0EKJNOBUDSMQ43DUFALJYXNJPQPVXF4NSZW


In [17]:
df['Neighbourhood'].iloc[0]

'Parkwoods'

In [18]:
neighborhood_latitude = df['Latitude'].iloc[23] # neighborhood latitude value
neighborhood_longitude = df['Longitude'].iloc[23] # neighborhood longitude value

neighborhood_name = df['Neighbourhood'].iloc[23] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Leaside are 43.7124, 43.7124.


In [19]:
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, neighborhood_latitude, neighborhood_longitude, VERSION, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?client_id=I3C1JCMVYJ5C0RGHF2VVQH31BZY4EOOFNPZ2Z1OM1SUFHAV0&client_secret=VJ3WUI2BTMYCV0EKJNOBUDSMQ43DUFALJYXNJPQPVXF4NSZW&ll=43.7124,43.7124&v=20180605&radius=500&limit=100'

In [20]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '604e6bb5ccb5ce3c759e14a8'},
 'response': {'groups': [{'items': [],
    'name': 'recommended',
    'type': 'Recommended Places'}],
  'headerFullLocation': 'Current map view',
  'headerLocation': 'Current map view',
  'headerLocationGranularity': 'unknown',
  'suggestedBounds': {'ne': {'lat': 43.716900004500005,
    'lng': 43.71861401917888},
   'sw': {'lat': 43.7078999955, 'lng': 43.706185980821125}},
  'totalResults': 0,
  'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."}}}

In [21]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [ ]:
toronto_venues = getNearbyVenues( df['Neighbourhood'],df['Latitude'], df['Longitude'])

In [23]:
toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Queen's Park, Ontario Provincial Government",43.6641,43.6641,"Горячий источник ""Гедуко""",43.664406,43.663278,Hot Spring
1,"Islington Avenue, Humber Valley Village",43.6662,43.6662,"Горячий источник ""Гедуко""",43.664406,43.663278,Hot Spring
2,"Dufferin, Dovercourt Village",43.6655,43.6655,"Горячий источник ""Гедуко""",43.664406,43.663278,Hot Spring
3,"High Park, The Junction South",43.6605,43.6605,"Горячий источник ""Гедуко""",43.664406,43.663278,Hot Spring
4,"University of Toronto, Harbord",43.6629,43.6629,"Горячий источник ""Гедуко""",43.664406,43.663278,Hot Spring


In [24]:
toronto_venues.shape

(6, 7)

In [27]:
toronto_venues.groupby("Neighborhood").count

<bound method DataFrameGroupBy.count of <pandas.core.groupby.generic.DataFrameGroupBy object at 0x7f11395a73d0>>

In [28]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighborhood,Hot Spring
0,"Queen's Park, Ontario Provincial Government",1
1,"Islington Avenue, Humber Valley Village",1
2,"Dufferin, Dovercourt Village",1
3,"High Park, The Junction South",1
4,"University of Toronto, Harbord",1


In [29]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Hot Spring
0,Church and Wellesley,1
1,"Dufferin, Dovercourt Village",1
2,"High Park, The Junction South",1
3,"Islington Avenue, Humber Valley Village",1
4,"Queen's Park, Ontario Provincial Government",1
5,"University of Toronto, Harbord",1


In [31]:
kclusters = 1

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0], dtype=int32)